In [75]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import requests
from tensorflow.python.framework import ops
import os.path
import csv


import random
import itertools
from itertools import cycle
import pandas as pd
import numpy as np
from scipy import interp, stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", color_codes=True)
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)
import sklearn.ensemble as ske
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import *
import sklearn.preprocessing as prep

import xgboost
import tensorflow as tf

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100) 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [76]:
ops.reset_default_graph()

# Create graph
def min_max_scale(data):
    preprocessor = prep.MinMaxScaler().fit(data)
    data = preprocessor.transform(data)
    return data

def standard_scale(data):
    preprocessor = prep.StandardScaler().fit(data)
    data = preprocessor.transform(data)
    return data

def get_random_block_from_data(x, y, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return (x[start_index:(start_index + batch_size)], y[start_index:(start_index + batch_size)])
sess = tf.Session()

In [77]:
folder_path = "/Users/prasanna/Downloads/"
file_name = "DS_Tech_Review_Dataset (1).txt"
data = pd.read_csv(folder_path + file_name, sep="|")
#data.describe()
#data.corr()
age_cols = [col for col in data.columns.values if "AGE" in col]
relevant_age_cols = [col for col in data.columns.values if "AGE" in col and ("_" in col or "UP" in col)]
data.dropna(thresh=len(data)*0.1, axis=1, inplace=True)

same_value_cols =[]
for col in data.columns.values:
    unique_cols = data[col].unique()
    if len(unique_cols) < 30 :
        #print (col, unique_cols)
        pass
    if len(unique_cols) == 1 or (len(unique_cols) == 2 and np.isnan(unique_cols).any()) :
        same_value_cols.append(col)

data["MAJOR_CREDIT_CARD_LIF"] = np.where(data["MAJOR_CREDIT_CARD_LIF"].isnull(),"NA", data["MAJOR_CREDIT_CARD_LIF"])
data.fillna(0,inplace=True)
data = pd.get_dummies(data, columns=["product", "MAJOR_CREDIT_CARD_LIF"])
data.drop(same_value_cols, axis=1, inplace=True)

target_values = data["target"].astype('float32').copy()
data.drop("target", axis=1, inplace=True)
class_names = ["No","Yes"]
features_names = data.columns.values.tolist()
features_data = data.astype('float32').as_matrix()
features_data.shape

(300000, 110)

In [78]:
features_data = min_max_scale(features_data)

In [6]:
# Declare batch size
batch_size = 25

# Initialize placeholders
x_data = tf.placeholder(shape=[None, 110], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Create variables for linear regression
A = tf.Variable(tf.random_normal(shape=[110,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)

# Declare loss function (Cross Entropy loss)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_output, labels=y_target))

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

In [8]:

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# Actual Prediction
prediction = tf.round(tf.sigmoid(model_output))
predictions_correct = tf.cast(tf.equal(prediction, y_target), tf.float32)
accuracy = tf.reduce_mean(predictions_correct)


target_values_re = target_values.reshape(300000,1)

# Training loop
loss_vec = []
train_acc = []
test_acc = []
for i in range(100):
    sess.run(train_step, feed_dict={x_data: features_data, y_target: target_values_re})

    temp_loss = sess.run(loss, feed_dict={x_data: features_data, y_target: target_values_re})
    loss_vec.append(temp_loss)
    temp_acc_train = sess.run(accuracy, feed_dict={x_data: features_data, y_target: np.transpose([target_values_re])})
    train_acc.append(temp_acc_train)
    temp_acc_test = sess.run(accuracy, feed_dict={x_data: features_data, y_target: np.transpose([target_values_re])})
    test_acc.append(temp_acc_test)
    if (i+1)%300==0:
        print('Loss = ' + str(temp_loss))

ValueError: Cannot feed value of shape (1, 300000, 1) for Tensor 'Placeholder_1:0', which has shape '(?, 1)'

In [31]:
features_data = min_max_scale(features_data)

In [42]:
ones_feat = []
zeros_feat = []
ones_target = []
zeros_target = []

for index in range(0, len(target_values)) :
    if target_values[index] == 0:
        zeros_feat.append(features_data[index])
        zeros_target.append(0)
    else :
        ones_feat.append(features_data[index])
        ones_target.append(1)        
Counter(ones_target)

Counter({1: 7004})

In [43]:
features_data2 = ones_feat
features_data2.extend(zeros_feat[0:7004])
target_values2 = ones_target
target_values2.extend(zeros_target[0:7004])
len(features_data2)
features_data2 = np.array(features_data2)
target_values2 = np.array(target_values2)
features_data2.shape

(14008, 110)

In [71]:
import tensorflow as tf
import numpy as np
import uuid

x = tf.placeholder(shape=[None, 3], dtype=tf.float32)
nn = tf.layers.dense(x, 3, activation=tf.nn.sigmoid)
nn = tf.layers.dense(nn, 5, activation=tf.nn.sigmoid)
encoded = tf.layers.dense(nn, 2, activation=tf.nn.sigmoid)
nn = tf.layers.dense(encoded, 5, activation=tf.nn.sigmoid)
nn = tf.layers.dense(nn, 3, activation=tf.nn.sigmoid)

cost = tf.reduce_mean((nn - x)**2)
optimizer = tf.train.RMSPropOptimizer(0.01).minimize(cost)
init = tf.global_variables_initializer()

tf.summary.scalar("cost", cost)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    uniq_id = "/tmp/tensorboard-layers-api/" + uuid.uuid1().__str__()[:6]
    summary_writer = tf.summary.FileWriter(uniq_id, graph=tf.get_default_graph())
    x_vals = np.random.normal(0, 1, (10000, 3))
    for step in range(1000):
        _, val, summary = sess.run([optimizer, cost, merged_summary_op],
                                   feed_dict={x: x_vals})
        if step % 5 == 0:
            print("step: {}, value: {}".format(step, val))
            summary_writer.add_summary(summary, step)
            
            

step: 0, value: 1.2717570066452026
step: 5, value: 1.2693437337875366
step: 10, value: 1.2662299871444702
step: 15, value: 1.2622262239456177
step: 20, value: 1.2571115493774414
step: 25, value: 1.2506248950958252
step: 30, value: 1.2424935102462769
step: 35, value: 1.2324697971343994
step: 40, value: 1.2203888893127441
step: 45, value: 1.206248164176941
step: 50, value: 1.190273642539978
step: 55, value: 1.1729271411895752
step: 60, value: 1.1548657417297363
step: 65, value: 1.1368099451065063
step: 70, value: 1.1194250583648682
step: 75, value: 1.1032216548919678
step: 80, value: 1.088534951210022
step: 85, value: 1.0755329132080078
step: 90, value: 1.0642427206039429
step: 95, value: 1.0546029806137085
step: 100, value: 1.0464880466461182
step: 105, value: 1.0397385358810425
step: 110, value: 1.0341801643371582
step: 115, value: 1.029640793800354
step: 120, value: 1.0259612798690796
step: 125, value: 1.022991418838501
step: 130, value: 1.0206104516983032
step: 135, value: 1.01871180

In [73]:
# tf_x = tf.placeholder(tf.float32, features_data.shape)     # input x
# tf_y = tf.placeholder(tf.int32, target_values.shape)  
tf_x = tf.placeholder(tf.float32, features_data.shape)     # input x
tf_y = tf.placeholder(tf.int32, target_values.shape)  

# tf_x = tf.placeholder(tf.float32, [batch_size, 784])     # input x
# tf_y = tf.placeholder(tf.float32, [batch_size, 2])

In [72]:
# a = 7004.0 
# b = 300000 - a
# c = b/float(a)
# c
# class_weights = []
# for item in target_values:
#     if item == 0:
#         class_weights.append(1.0)
#     if item == 1:
#         class_weights.append(20.0)
# class_weights = np.array(class_weights)
# Counter(class_weights)


sample_weights_data =  target_values.copy()
sample_weights_data[sample_weights_data == 1] = 40
sample_weights_data[sample_weights_data == 0] = 1
Counter(sample_weights_data), Counter(target_values)

(Counter({1.0: 292996, 40.0: 7004}), Counter({0.0: 292996, 1.0: 7004}))

In [74]:
tf_y.shape

TensorShape([Dimension(300000)])

In [65]:
# l1 = tf.layers.dense(tf_x, 10, tf.nn.relu)          # hidden layer
l1 = tf.layers.dense(tf_x, 30, tf.nn.sigmoid)          # hidden layer
l2 = tf.layers.dense(l1, 10, tf.nn.sigmoid)
output = tf.layers.dense(l2, 2)                     # output layer

loss = tf.losses.sparse_softmax_cross_entropy(labels=tf_y, logits=output, weights = class_weights)           # compute cost
accuracy = tf.metrics.accuracy(          # return (acc, update_op), and create 2 local variables
    labels=tf.squeeze(tf_y), predictions=tf.argmax(output, axis=1),)[1]
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss)

sess = tf.Session()                                                                 # control training and others
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)     # initialize var in graph

for step in range(500):
    _, acc, pred = sess.run([train_op, accuracy, output], {tf_x: features_data, tf_y: target_values})
    if step % 100 == 0:
        print (acc)

0.0233467
0.91894
0.947653
0.953454
0.936823


In [69]:
cf_arr = []
for step in range(500):
    _, acc, pred = sess.run([train_op, accuracy, output], {tf_x: features_data, tf_y: target_values})
    if step % 100 == 0:
        print (acc)
        pred_max = np.argmax(pred, axis=1)
        cf_current = confusion_matrix(target_values, pred_max)
        print (cf_current)
        cf_arr.append(cf_current)

0.83699
[[245227  47769]
 [  3078   3926]]
0.836883
[[245207  47789]
 [  3064   3940]]
0.836777
[[245138  47858]
 [  3055   3949]]
0.836673
[[245120  47876]
 [  3049   3955]]
0.836571
[[245919  47077]
 [  3081   3923]]


In [70]:
Counter(pred_max)

Counter({0: 249000, 1: 51000})

In [ ]:
[[245542  47454]
 [  3102   3902]]

In [67]:
pred_max = np.argmax(pred, axis=1)
from collections import Counter
Counter(pred_max)
from sklearn.metrics import *
confusion_matrix(target_values, pred_max)


array([[245348,  47648],
       [  3105,   3899]])

In [61]:
pred_max = np.argmax(pred, axis=1)
from collections import Counter
Counter(pred_max)

Counter({0: 171254, 1: 128746})

array([[170158, 122838],
       [  1096,   5908]])

In [18]:
pred[0:5]

array([[ 1.66, -1.66],
       [ 1.66, -1.66],
       [ 1.66, -1.66],
       [ 1.66, -1.66],
       [ 1.66, -1.66]], dtype=float32)

In [36]:
from scipy import stats
stats.describe(target_values)

DescribeResult(nobs=300000, minmax=(0.0, 1.0), mean=0.023346666, variance=0.022801677, skewness=6.313205718994141, kurtosis=37.85655854301618)

In [25]:
from collections import Counter
Counter(pred_max)

Counter({0: 299988, 1: 12})

In [26]:
Counter(target_values)

Counter({0.0: 292996, 1.0: 7004})